**Import needed packages/modules**

In [ ]:
# Cell 1
import matplotlib.pyplot as plt
import numpy as np
from numba import njit

**Define a function that converts HSV color triplets to RGB color triplets**

In [ ]:
# Cell 2
@njit
def hsv_to_rgb(h, s, v):
    if s == 0:
        # If saturation is 0, the color is a shade of gray
        return int(v * 255), int(v * 255), int(v * 255)

    # Normalize hue to be within 0 and 360
    h = h % 360

    # Sector calculations for determining RGB components
    sector_pos = h / 60.0
    sector_index = int(sector_pos)
    fractional_part = sector_pos - sector_index

    # Calculate intermediate values for RGB components
    p = v * (1 - s)
    q = v * (1 - (s * fractional_part))
    t = v * (1 - (s * (1 - fractional_part)))

    # Convert fractional values to scale of 0 to 255
    r, g, b = 0, 0, 0
    if sector_index == 0:
        r, g, b = v, t, p
    elif sector_index == 1:
        r, g, b = q, v, p
    elif sector_index == 2:
        r, g, b = p, v, t
    elif sector_index == 3:
        r, g, b = p, q, v
    elif sector_index == 4:
        r, g, b = t, p, v
    elif sector_index == 5:
        r, g, b = v, p, q

    return int(r * 255), int(g * 255), int(b * 255)

**Define a function to calculate the <u>Mandelbrot Set</u> within a given world rectangle**
1. The function receives a `numpy array` representing a virtual "screen" of pixels
2. The function also receives a single tuple containing two tuples of $(x,y)$ Cartesian coordinates
3. The $1^{st}$ tuple is the $(x_{min},\;y_{min}$) world coordinates (lower-left)
4. The $2^{nd}$ tuple is the $(x_{max},\;y_{max}$) world coordinates (upper-right)

In [ ]:
# Cell 3
@njit
def calculate_mandelbrot_set(scr, world_rect):
    # Array shapes are ROW x COL, so the x size is the 2nd element
    # Also, we subtract one because array indexes start at 0
    screen_x_max = scr.shape[1] - 1
    screen_y_max = scr.shape[0] - 1

    world_x_min = world_rect[0][0]
    world_y_min = world_rect[0][1]
    world_x_max = world_rect[1][0]
    world_y_max = world_rect[1][1]
    world_x_size = world_x_max - world_x_min
    world_y_size = world_y_max - world_y_min

    radius = 16
    max_iter = 100

    for sx in range(screen_x_max):
        for sy in range(screen_y_max):
            wx = world_x_min + world_x_size * sx / screen_x_max
            wy = world_y_max - world_y_size * sy / screen_y_max

            c = complex(wx, wy)
            z = complex(0, 0)

            iter = 0
            while abs(z) < radius and iter < max_iter:
                z = z * z + c
                iter += 1

            # Convert HSV color to RGB color (saturation @ 100%)
            hue = iter / max_iter * 360
            value = 1 if iter < max_iter else 0
            rgb = hsv_to_rgb(hue, 1.0, value)

            # Set "pixel" (an element in scr matrix) to the color
            if 0 <= sx <= screen_x_max and 0 <= sy <= screen_y_max:
                scr[sy, sx] = rgb

**Define a function that draws the <u>Mandelbrot Set</u> within a given world rectangle**

In [ ]:
# Cell 4
def draw_mandelbrot_set(world_rect):
    scr = np.zeros((600, 600, 3), dtype=np.uint8)
    calculate_mandelbrot_set(scr, world_rect)
    plt.figure(figsize=(10, 10))
    plt.imshow(scr, interpolation="nearest", aspect="equal")
    plt.axis("off")
    plt.show()

**View the Mandelbrot Set from $(-2.2-1.51i)$ to $(1.0+1.51i)$**

In [ ]:
# Cell 5
draw_mandelbrot_set(((-2.2, -1.51), (1.0, 1.51)))

**View the Mandelbrot Set from $(-0.4950+0.3981i)$ to $(0.3486+1.1942i)$**

In [ ]:
# Cell 6
draw_mandelbrot_set(((-0.4950, 0.3981), (0.3486, 1.1942)))

**View the Mandelbrot Set from $(-0.1981+0.9956i)$ to $(-0.1234+1.0661i)$**

In [ ]:
# Cell 7
draw_mandelbrot_set(((-0.1981, 0.9956), (-0.1234, 1.0661)))

**View the Mandelbrot Set from $(-1.3849-0.0218i)$ to $(-1.1784+0.1730i)$**

In [ ]:
# Cell 8
draw_mandelbrot_set(((-1.3849, -0.0218), (-1.1784, 0.1730)))

**View the Mandelbrot Set from $(-1.2306+0.1547i)$ to $(-1.2238+0.1611i)$**

In [ ]:
# Cell 9
draw_mandelbrot_set(((-1.2306, 0.1547), (-1.2238, 0.1611)))